In [2]:
!pip install numba xgboost lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 16.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 61.6 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split

2024-08-18 19:05:58.134997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 19:05:58.135137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 19:05:58.216023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-18 19:05:58.406500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-18 19:05:59.805720: W tensorflow/compiler/tf2

In [4]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Activation, SpatialDropout1D, \
    LayerNormalization, Add, Input, GlobalAveragePooling1D
import tensorflow.keras.backend as kb

In [5]:
from numba import cuda 
device = cuda.get_current_device()

In [6]:
data = pd.read_csv("./data/Germany_20140101_20231231.csv")

### Logging helper

In [7]:
import os
def log(metrics):
    """
    Log the evaluation metrics to a file.

    Parameters:
    - metrics: Dictionary containing evaluation metrics
    """
    # Create the directory structure if it doesn't exist
    model_name = metrics['model_name']
    window_size = metrics['window_size']

    # Define the output path
    output_dir = 'results'
    output_path = os.path.join('.', output_dir, model_name, "metrics.csv")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Check if the file already exists
    file_exists = os.path.isfile(output_path)

    # Write the metrics to the file (append mode)
    with open(output_path, 'a') as f:
        # Write the header only if the file does not already exist
        if not file_exists:
            f.write(f"model_name,window_size,rmse,mae,smape,r2,forecast_bias,training_time\n")
        f.write(f"{model_name},{window_size},{metrics['rmse']},{metrics['mae']},{metrics['smape']},{metrics['r2']},{metrics['forecast_bias']},{metrics['training_time']}\n")

    print(f"Metrics have been logged to: {output_path}")

### Data Loading and Preprocessing

In [8]:
def preprocess_df(df):
    # Ensure the DataFrame is sorted by date
    df = df.sort_index()

    # After some EDA we decided to drop these columns
    cols_to_drop = ['stations', 'description', 'conditions', 'icon', 'severerisk']
    df = df.drop(cols_to_drop, axis=1)

    # null values in 'sealevelpressure' column are filled with the mean value
    df['sealevelpressure'] = df['sealevelpressure'].fillna(df['sealevelpressure'].mean())

    # null values in 'visibility' column are filled with the mean value
    df['visibility'] = df['visibility'].fillna(df['visibility'].mean())

    # null values in 'windgust' column are filled with the 'windspeed' values
    df['windgust'] = df['windgust'].fillna(df['windspeed'])

    # null values in 'preciptype' column are filled with the 'none'
    df['preciptype'] = df['preciptype'].fillna('none')

    # 'preciptype' is a categorical feature, so we'll one-hot encode it
    df = pd.get_dummies(df, columns=['preciptype'], drop_first=True)

    # 'name' is a categorical feature, so we'll one-hot encode it
    df = pd.get_dummies(df, columns=['name'], drop_first=True)

    # Convert 'datetime' column to datetime if it's not already
    if 'datetime' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['datetime']):
        df['datetime'] = pd.to_datetime(df['datetime'])

    # Set 'datetime' as index if it's not already
    if 'datetime' in df.columns:
        df = df.set_index('datetime')

    # Handle 'sunrise' and 'sunset' columns
    if 'sunrise' in df.columns and 'sunset' in df.columns:
        # Convert to datetime
        df['sunrise'] = pd.to_datetime(df['sunrise'])
        df['sunset'] = pd.to_datetime(df['sunset'])

        # Extract time features
        df['sunrise_hour'] = df['sunrise'].dt.hour + df['sunrise'].dt.minute / 60
        df['sunset_hour'] = df['sunset'].dt.hour + df['sunset'].dt.minute / 60

        # Calculate day length in hours
        df['day_length'] = (df['sunset'] - df['sunrise']).dt.total_seconds() / 3600

        # Drop original 'sunrise' and 'sunset' columns
        df = df.drop(['sunrise', 'sunset'], axis=1)

    return df

In [9]:
def split_time_series_data(df, target_column, val_size=0.15, test_size=0.15, window_size=1):
    """
    Split a time series DataFrame into training, validation, and test sets.

    Parameters:
    - df: pandas DataFrame containing the time series data
    - target_column: string, name of the column to be predicted
    - val_size: float, proportion of data to use for validation (default 0.15)
    - test_size: float, proportion of data to use for testing (default 0.15)
    - window_size: int, size of the sliding window for feature creation (default 1)

    Returns:
    - x_train, y_train: Training data and labels as DataFrames
    - x_val, y_val: Validation data and labels as DataFrames
    - x_test, y_test: Test data and labels as DataFrames
    """

    # Preprocess the DataFrame
    df = preprocess_df(df)

    # Calculate sizes
    train_size = 1 - val_size - test_size
    total_size = len(df)
    train_end = int(total_size * train_size)
    val_end = int(total_size * (train_size + val_size))

    # Split the data
    train_data = df.iloc[:train_end]
    val_data = df.iloc[train_end:val_end]
    test_data = df.iloc[val_end:]

    # Function to create features and labels
    def create_features(data):
        features = []
        labels = []
        for i in range(len(data) - window_size):
            features.append(data.iloc[i:i + window_size].drop(columns=[target_column]))
            labels.append(data.iloc[i + window_size][target_column])
        return pd.concat(features, keys=range(len(features))), pd.Series(labels, name=target_column)

    # Create features and labels for each set
    x_train, y_train = create_features(train_data)
    x_val, y_val = create_features(val_data)
    x_test, y_test = create_features(test_data)

    return x_train, y_train, x_val, y_val, x_test, y_test

def prepare_for_rnn(df, target_column, val_size=0.15, test_size=0.15, window_size=1):
    # Preprocess the DataFrame
    df = preprocess_df(df)

    # Calculate sizes
    train_size = 1 - val_size - test_size
    total_size = len(df)
    train_end = int(total_size * train_size)
    val_end = int(total_size * (train_size + val_size))

    # Split the data
    train_data = df.iloc[:train_end]
    val_data = df.iloc[train_end:val_end]
    test_data = df.iloc[val_end:]

    # process the data
    def process_data(data):
        data = data[target_column].values.reshape(-1, 1)
        x_ = []
        y_ = []
        for i in range(window_size, len(data)):
            x_.append(data[i - window_size:i, 0])
            y_.append(data[i, 0])
        return np.array(x_), np.array(y_)

    # for training data (scaling and fittransform, the others will be transformed only)
    scaper = StandardScaler()
    x_train, y_train = process_data(train_data)
    x_train = scaper.fit_transform(x_train)

    # for validation data
    x_val, y_val = process_data(val_data)
    x_val = scaper.transform(x_val)

    # for test data
    x_test, y_test = process_data(test_data)
    x_test = scaper.transform(x_test)

    # reshape the input data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return x_train, y_train, x_val, y_val, x_test, y_test

In [10]:
def prepare_for_gb_models(df, target, window_size):
    x, y = [], []
    for i in range(len(df) - window_size):
        x.append(df.drop(columns=[target]).iloc[i: i+ window_size])
        y.append(df.iloc[i + window_size][target])
    x, y = np.array(x), np.array(y)
    n_samples, window, n_features = x.shape
    x = x.reshape((n_samples, window * n_features))
    x_train, y_train, x_test, y_test = train_test_split(x, y, test_size=0.3)

### Data Model Preparation

In [11]:
def prepare_for_model(
        model_name, model_func,
        x_train, y_train,
        x_val, y_val,
        x_test, y_test,
        window_size=7):
    """
    Prepare the model and data for training and evaluation. completely manual

    Parameters:
    - model_name: Name of the model
    - model_func: Function to create the model
    - x_train, y_train: Training data
    - x_val, y_val: Validation data
    - x_test, y_test: Testing data
    - window_size: Size of the sliding window used

    Returns:
    - model: The model
    - x_train, y_train: Training data and labels
    - x_val, y_val: Validation data and labels
    - x_test, y_test: Testing data and labels
    """
    if model_name == 'rnn':
        # build the model and rturn the data as ism since they were already processed in prepare_for_rnn
        model = model_func((window_size, x_train.shape[2]))
        print("Data and model prepared for RNN")
        return model, x_train, y_train, x_val, y_val, x_test, y_test

    feature_names = x_train.columns.tolist()
    model = None  # Placeholder override by the model_func

    input_shape = (window_size, len(feature_names))
    print(f"Input shape for {model_name}: {input_shape}")

    # Convert DataFrames to numpy arrays and reshape for RNN models
    x_train = x_train.values.reshape(-1, window_size, len(feature_names)).astype(np.float32)
    x_val = x_val.values.reshape(-1, window_size, len(feature_names)).astype(np.float32)
    x_test = x_test.values.reshape(-1, window_size, len(feature_names)).astype(np.float32)

    model = model_func(input_shape)
    print(f"Data prepared for {model_name} model")
    return model, x_train, y_train, x_val, y_val, x_test, y_test

### Model Training and Evaluation

In [12]:
def train_and_evaluate_model(
        model,
        model_name,
        x_train, y_train,
        x_val, y_val,
        x_test, y_test,
        window_size,
        is_deep_learning=False,
        epochs=100,
        batch_size=32,
        hyperparams=None):
    """
    Train the model, optionally perform hyperparameter tuning, and evaluate it.

    Parameters:
    - model: The machine learning model to be trained
    - model_name: Name of the model
    - X_train, y_train: Training data
    - X_val, y_val: Validation data
    - X_test, y_test: Testing data
    - window_size: Size of the sliding window used
    - is_deep_learning: Boolean indicating if it's a deep learning model
    - epochs, batch_size: Parameters for deep learning models
    - hyperparams: Dictionary of hyperparameters for traditional ML model tuning

    Returns:
    - model: The trained model
    - metrics: Dictionary containing evaluation metrics
    """
    start_time = time.time()

    if is_deep_learning:
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        model_checkpoint = ModelCheckpoint(f"experiment_models/{model_name}_best_model.keras", save_best_only=True)
        history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                            epochs=epochs, batch_size=batch_size,
                            callbacks=[early_stopping, model_checkpoint])
    elif hyperparams is not None:


        tscv = TimeSeriesSplit(n_splits=5)
        grid_search = GridSearchCV(model, hyperparams, cv=tscv, scoring='neg_mean_squared_error')
        grid_search.fit(x_train, y_train)
        model = grid_search.best_estimator_

    else:
        model.fit(x_train, y_train)

    training_time = time.time() - start_time

    # Make predictions
    y_pred = model.predict(x_test)

    # Ensure y_pred is 1D
    y_pred = y_pred.flatten()
    y_test = y_test.flatten()

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    smape = 100 * np.mean(2 * np.abs(y_pred - y_test) / (np.abs(y_pred) + np.abs(y_test)))
    r2 = r2_score(y_test, y_pred)
    forecast_bias = np.mean(y_pred - y_test)

    # Compile metrics dictionary
    metrics = {
        'model_name': model_name,
        'window_size': window_size,
        'rmse': rmse,
        'mae': mae,
        'smape': smape,
        'r2': r2,
        'forecast_bias': forecast_bias,
        'training_time': training_time
    }
    return model, metrics

### Model Creation

In [13]:
# ------------------ RNN Model ------------------
def create_rnn_model(input_shape, output_units=1):
    model = Sequential([
        Input(shape=input_shape),
        SimpleRNN(64, return_sequences=True),
        SimpleRNN(32, return_sequences=True),
        SimpleRNN(16),
        Dense(output_units)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model


# ------------------ LSTM Model ------------------
def create_lstm_model(input_shape, output_units=1):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        LSTM(32),
        Dense(output_units)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model


# ------------------ CNN Model ------------------
def create_cnn_model(input_shape, output_units=1):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling1D(pool_size=2, padding='same'),
        Conv1D(filters=32, kernel_size=2, activation='relu', padding='same'),
        GlobalAveragePooling1D(),
        Dense(32, activation='relu'),
        Dense(output_units)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model


# ------------------ LightGBM Model ------------------
def create_lightgbm_model():
    return lgb.LGBMRegressor()


# ------------------ XGBoost Model ------------------
def create_xgboost_model():
    return xgb.XGBRegressor()


# ------------------ TCM Model ------------------
def causal_padding(x):
    return kb.temporal_padding(x, (1, 0))


def residual_block(x, dilation_rate, nb_filters, kernel_size, dropout_rate=0.1):
    prev_x = x
    x = LayerNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size,
               dilation_rate=dilation_rate, padding='causal')(x)
    x = LayerNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size,
               dilation_rate=dilation_rate, padding='causal')(x)
    x = Add()([prev_x, x])
    return x


def create_tcn_model(input_shape, output_units=1, nb_filters=64, kernel_size=2,
                     nb_stacks=1, dilations=[1, 2, 4, 8], dropout_rate=0.2):
    input_layer = Input(shape=input_shape)

    x = Conv1D(nb_filters, kernel_size, padding='causal', name='initial_conv')(input_layer)

    for _ in range(nb_stacks):
        for dilation_rate in dilations:
            x = residual_block(x, dilation_rate, nb_filters,
                               kernel_size, dropout_rate)

    x = Activation('relu')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(output_units)(x)

    model = Model(inputs=[input_layer], outputs=[x])
    model.compile(optimizer='adam', loss='mse')

    return model

### Experimentation

In [14]:
def do_job(df, target_column, models, results, window_sizes=None):
    
    if window_sizes is None: window_sizes = [7]
    val_size = 0.15
    test_size = 0.15
    
    for model_name, model_func in models.items():
        for window_size in window_sizes:
            
            # check if the model is already trained
            trainingdone = False
            if len(results[model_name]) > 0:
                for key, item in results[model_name].items():
                    if f"{model_name}_{window_size}" in key:
                        print(f"{model_name} with window size of {window_size} is already trained.")
                        trainingdone = True
            if trainingdone:
                continue
            
            print(f"Training and evaluating {model_name}... with window size of {window_size}")
            
            try:
                x_train: None
                y_train: None
                x_val: None
                y_val: None
                x_test: None
                y_test: None
                deep_learning = True
                epochs = 20
                
                if model_name == "rnn":
                    x_train, y_train, x_val, y_val, x_test, y_test = prepare_for_rnn(
                        df, target_column, val_size, test_size, window_size
                    )
                else:
                    x_train, y_train, x_val, y_val, x_test, y_test = split_time_series_data(
                        df, target_column, val_size, test_size, window_size
                    )
                
                model, x_train, y_train, x_val, y_val, x_test, y_test = prepare_for_model(
                    model_name, model_func, 
                    x_train, y_train, 
                    x_val, y_val, 
                    x_test, y_test,
                    window_size
                )
                if model_name != "rnn":
                    y_train = y_train.values
                    y_val = y_val.values
                    y_test = y_test.values
                    epochs = 100
                    
                trained_model, metrics = train_and_evaluate_model(
                    model, model_name, x_train, y_train, x_val, y_val, x_test, y_test, 
                    window_size=window_size, is_deep_learning=deep_learning, epochs=epochs
                )
                
                log(metrics)
                
                results[model_name][f"{model_name}_{window_size}"] = metrics
                
                
            except Exception as e:
                print(f"Error occurred while training {model_name}: {str(e)}")
                results[model_name]["metrics"][f"{model_name}_{window_size}"] = "error" + str(e)
    return results

In [15]:
models = {
    "cnn": create_cnn_model,
    "lstm": create_lstm_model,
    "tcn": create_tcn_model,
    "rnn": create_rnn_model
}

In [16]:
results = {
    "cnn": { },
    "lstm": { },
    "tcn": { },
    "rnn": { }
}

In [17]:
windows_sizes = [7, 14, 30, 60, 180, 365]

In [18]:
results = do_job(data, "temp", models, results, windows_sizes)

Training and evaluating cnn... with window size of 7
Input shape for cnn: (7, 34)


2024-08-18 19:06:09.334714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-18 19:06:09.637246: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-18 19:06:09.637416: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Data prepared for cnn model
Epoch 1/100


2024-08-18 19:06:14.816214: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-18 19:06:17.620177: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f168b783510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-18 19:06:17.620189: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2024-08-18 19:06:17.638123: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1724007977.752681     127 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


320/320 [==============================] - 9s 4ms/step - loss: 21.2786 - val_loss: 9.2797
Epoch 2/100
320/320 [==============================] - 1s 3ms/step - loss: 7.4111 - val_loss: 9.0902
Epoch 3/100
320/320 [==============================] - 1s 3ms/step - loss: 6.1649 - val_loss: 5.1519
Epoch 4/100
320/320 [==============================] - 1s 3ms/step - loss: 5.5795 - val_loss: 5.7701
Epoch 5/100
320/320 [==============================] - 1s 3ms/step - loss: 5.6096 - val_loss: 4.5134
Epoch 6/100
320/320 [==============================] - 1s 3ms/step - loss: 5.4505 - val_loss: 5.1950
Epoch 7/100
320/320 [==============================] - 1s 3ms/step - loss: 5.1866 - val_loss: 4.1690
Epoch 8/100
320/320 [==============================] - 1s 3ms/step - loss: 4.8805 - val_loss: 4.0412
Epoch 9/100
320/320 [==============================] - 1s 3ms/step - loss: 4.8635 - val_loss: 4.7083
Epoch 10/100
320/320 [==============================] - 1s 3ms/step - loss: 4.6236 - val_loss: 3.8360


In [19]:
for model_name, model_results in results.items():
    print(f"Results for {model_name}")
    for key, item in model_results.items():
        if "Error" in key:
            print(item)
        else:
            for metric_name, metric_value in item.items():
                if metric_name != "model_name":
                    print(f"{metric_name}: {metric_value}")
        print("___")
    print("=======")

Results for cnn
window_size: 7
rmse: 1.8438046530812073
mae: 1.4263358697929425
smape: 20.67530451959958
r2: 0.9373058439973899
forecast_bias: 0.48628158737101873
training_time: 51.649333000183105
___
window_size: 14
rmse: 1.942795476353371
mae: 1.5226419753081157
smape: 21.490223530904046
r2: 0.9304391709262545
forecast_bias: -0.6809481710435
training_time: 33.48738694190979
___
window_size: 30
rmse: 1.781208494605541
mae: 1.3871125896799894
smape: 20.788647242261003
r2: 0.9415457091053503
forecast_bias: 0.030387294238173053
training_time: 49.61276650428772
___
window_size: 60
rmse: 1.7546349849888667
mae: 1.3493007248816191
smape: 19.818515055854586
r2: 0.9418602905740197
forecast_bias: -0.05552328635517363
training_time: 66.48372149467468
___
window_size: 180
rmse: 1.9885828220726796
mae: 1.5563020118365942
smape: 21.367518607271446
r2: 0.9256233130039209
forecast_bias: 0.6680741243138465
training_time: 47.68547487258911
___
window_size: 365
rmse: 1.882897563867991
mae: 1.4623371909